# 多标签文本分类流程
1. 数据预处理
a.中文分词
b.去停用词
2. 标签归一化
3. 特征提取
4. 模型训练

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertModel, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer
import os
import time


# 定义数据集类
class RecordDataset(Dataset):
    def __init__(self, text, label_matrix, tokenizer, max_len):
        self.text = text
        self.label_matrix = label_matrix
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, index):
        text = str(self.text[index])
        label = torch.tensor(self.label_matrix[index], dtype=torch.float)
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True,
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': label
        }


# 定义数据加载器
def create_data_loader(df, tokenizer, max_len, batch_size):
    '''
    @param df : dataframe数据集
    @param tokenizer : 分词器
    @param max_len : 最大长度
    @param batch_size : 批次大小
    '''
    # 分隔出训练集与测试集
    df_train = df.sample(frac=0.9, random_state=42) # 随机取90%的数据作为训练集
    df_test = df.drop(df_train.index) # 剩下的10%作为验证集

    # 检查训练集和测试集的标签，将标签合并为一个标签集合
    train_labels = set()
    for labels in df_train['label']:
        train_labels.update(str(labels).split('\\n'))
    test_labels = set()
    for labels in df_test['label']:
        test_labels.update(str(labels).split('\\n'))
    all_labels = list(train_labels | test_labels)

    mlb = MultiLabelBinarizer(classes=all_labels)
    label_list_train = [str(i).split('\\n') for i in df_train['label']]
    label_matrix = mlb.fit_transform(label_list_train)
    label_list_test = [str(i).split('\\n') for i in df_test['label']]
    test_label_matrix = mlb.transform(label_list_test)

    ds = RecordDataset(
        text=df_train['text'].to_numpy(),
        # label=df['label'].to_numpy(),
        label_matrix=label_matrix,
        tokenizer=tokenizer,
        max_len=max_len
    )

    test_dataset = RecordDataset(
        text=df_test['text'].to_numpy(),
        label_matrix=test_label_matrix,
        tokenizer=tokenizer,
        max_len=max_len
    )
    
    # 打印数据集信息
    print('Data set size:', len(ds))
    print('Label size:', len(label_matrix[1]))
    return DataLoader(ds,batch_size=batch_size,num_workers=4), DataLoader(test_dataset, batch_size=batch_size, num_workers=4)


# 定义训练函数
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    '''
    @param model : 模型
    @param data_loader : 数据加载器
    @param loss_fn : 损失函数
    @param optimizer : 优化器
    @param device : 设备
    @param scheduler : 动态学习率
    @param n_examples : 样本数
    @return : 准确率和损失
    '''
    model = model.train()
    
    losses = []
    correct_predictions = 0
    
    for d in data_loader:
        input_ids = d['input_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)
        labels = d['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        _, preds = torch.max(outputs.logits, dim=1)
        loss = loss_fn(outputs.logits, labels)
        
        result = outputs.logits
        threshold = 0.9
        for row in range(result.shape[0]):
            for col in range(result.shape[1]):
                if result[row][col] > threshold:
                    result[row][col] = 1
                else:
                    result[row][col] = 0
        
        correct_predictions += torch.sum(result == labels)
        losses.append(loss.item())
        
        # print(correct_predictions)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
    # return float(correct_predictions) / n_examples, np.mean(losses)
    return float(correct_predictions) / (n_examples * num_labels), np.mean(losses)


# 使用测试集验证模型
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model  = model.eval()
    losses = []
    corret_predictions = 0
    num_labels = len(data_loader.dataset.label_matrix[1])
    with torch.no_grad():
        for d in data_loader:
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            labels = d['labels'].to(device)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = loss_fn(outputs.logits, labels)
            losses.append(loss.item())

            # float转换为int
            result = outputs.logits
            threshold = 0.95
            for row in range(result.shape[0]):
                for col in range(result.shape[1]):
                    if result[row][col] > threshold:
                        result[row][col] = 1
                    else:
                        result[row][col] = 0
                    if result[row][col] == labels[row][col]:
                        corret_predictions += 1
    
    return float(corret_predictions) / (n_examples * num_labels), np.mean(losses)
            
            
def show_time(start_time, end_time):
    '''
    @param start_time : 开始时间
    @param end_time : 结束时间
    @return : 用时
    '''
    hours, rem = divmod(end_time - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    return "{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds)

# 清空cuda
torch.cuda.empty_cache()

EPOCHS = 10
BATCH_SIZE = 16
MAX_LEN = 128
LEARNING_RATE = 2e-5
# 模型路径
model_dir = 'BERT/chinese_bert_wwm_ext_L-12_H-768_A-12/'

# 数据读取
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
df_total = pd.read_csv(os.path.join('Record Collections', 'medical_record.csv'))
tokenizer = BertTokenizer.from_pretrained(model_dir)
train_data_loader, test_data_loader = create_data_loader(df_total, tokenizer, MAX_LEN, BATCH_SIZE)
num_labels = len(train_data_loader.dataset.label_matrix[1])

d:\dck\Softwares\anaconda\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\dck\Softwares\anaconda\envs\nlp\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(
Some weights of the model checkpoint at BERT/chinese_bert_wwm_ext_L-12_H-768_A-12/pytorch_model.bin/pytorch_model.bin were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expecte

Model: BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps

In [2]:
# 模型加载
config = BertConfig.from_pretrained(os.path.join(model_dir, 'pytorch_model.bin/config.json'))
model = BertForSequenceClassification.from_pretrained(os.path.join(model_dir, 'pytorch_model.bin/pytorch_model.bin'), config=config)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, correct_bias=False, eps=1e-8)
model.to(device)

print('Model:', model)
print('Parameters for each layer:', [p.numel() for p in model.parameters()])
print('Paramter count:', sum([p.numel() for p in model.parameters()]))

d:\dck\Softwares\anaconda\envs\nlp\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Data set size: 10
Label size: 28


In [3]:
loss_fn = nn.CrossEntropyLoss().to(device) # 损失函数定义
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_data_loader) * EPOCHS
) # 动态学习率

# 训练模型
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)
    start_time = time.time()
    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(train_data_loader.dataset)
    )
    end_time = time.time()
    print(f'{show_time(start_time, end_time)} : Train loss {train_loss} accuracy {train_acc}')

    # 验证模型
    test_acc, test_loss = eval_model(
        model,
        test_data_loader,
        loss_fn,
        device,
        len(test_data_loader)
    )
    print(f'{show_time(start_time, end_time)} : Test loss {test_loss} accuracy {test_acc}')

Epoch 1/2
----------
